In [1]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import time
import datetime

import scipy.stats as sts

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_json('table.jsonl', lines=True)

Gold = a * time + b

In [3]:
Y, X = data[data['type']=='interval'].dropna(subset=[['gold','time']])['gold'].as_matrix().reshape(-1, 1), data[data['type']=='interval'].dropna(subset=[['gold','time']])['time'].as_matrix().reshape(-1, 1)

In [4]:
def smodt(X,Y):
    X = sm.add_constant(X)
    mod = sm.OLS(Y, X)
    res = mod.fit()
    print('\n', res.summary())
    
def Testing_grid_reg(X_train, Y_train):
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    grid = {'fit_intercept':[True,False], 
            'normalize':[True,False], 
            'copy_X':[True, False]}
    
    clf = LinearRegression(n_jobs = 5)
    
    start_time = datetime.datetime.now()
    gs = GridSearchCV(clf, scoring='r2', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    lm = gs.best_estimator_
    
    lm.fit(X,Y)
    params = np.append(lm.intercept_,lm.coef_)
    predictions = lm.predict(X)
    
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
    MSE = (sum((X-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-sts.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]
    
    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)
    
    myDF3 = pd.DataFrame()
    myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilites"] = [params,sd_b,ts_b,p_values]
    
    print ('\n', myDF3)
    
    smodt(X,Y)
    return gs.best_estimator_ 

Значимы​​ ли​​ коэффициенты​​ регрессии?​​ Нужно​​ ли​​ использовать​​ intercept​​ в
данной​​ модели?​​ Какой​​ смысл​​ в​​ терминах​​ игры​​ имеет​​ коэффициент​​ наклона?

In [5]:
clf = Testing_grid_reg(X,Y)

Time elapsed: 0:00:00.174009
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': True, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': True, 'fit_intercept': False, 'normalize': False}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': True}
0.716 (+/-0.005) for {'copy_X': False, 'fit_intercept': True, 'normalize': False}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': True}
0.636 (+/-0.004) for {'copy_X': False, 'fit_intercept': False, 'normalize': False}

    Coefficients  Standard Errors  t values  Probabilites
0    -3867.9249           50.486   -76.614           0.0
1        5.8809            0.026   229.697           0.0

                             OLS Regression Results                            
Dep. Variable:                      y   R-squared:               

Оба коэффициента значимы - $|t_{value}| > P_{value} $. 

В начале игры(time = 0), золото(intercept) будет равно -3867.9249, что не отражает реальность, а также говорит о наличии других проблем (например, мультикорреляция). 

Коэффициент наклона говорит нам, что увеличение золота на одну единицу приводит к увеличению времени на 5.8809 сек, или каждые 5.8809 секунд приносят игроку 1 голд.

In [22]:
data[(data['type']=='DOTA_COMBATLOG_DEATH')].isnull().sum()[lambda x: x!= 3430]

attackerhero        0
attackerillusion    0
attackername        0
attackerteam        1
inflictor           0
location            0
sourcename          0
targethero          0
targetillusion      0
targetname          0
targetsourcename    0
targetteam          0
tick                0
time                0
type                0
value               0
dtype: int64

'attackername', 'targetname', 'time', 'location'

In [57]:
Combat = data[(data['type']=='DOTA_COMBATLOG_DEATH')][['attackername', 'targetname', 'time', 'location']].sort_values(by=['time'])

In [30]:
data.columns

Index(['assists', 'attackerhero', 'attackerillusion', 'attackername',
       'attackerteam', 'camps_stacked', 'creeps_stacked', 'deaths', 'denies',
       'ehandle', 'entityleft', 'firstblood_claimed', 'gold', 'gold_reason',
       'health', 'hero_id', 'hero_inventory', 'hero_stash', 'inflictor', 'key',
       'kills', 'level', 'lh', 'life_state', 'location', 'manacost',
       'max_health', 'max_mana', 'nearby_creep_death_count', 'obs_placed',
       'observers_placed', 'player1', 'player2', 'pred_vict', 'randomed',
       'roshans_killed', 'rune_pickups', 'sen_placed', 'slot', 'slow_duration',
       'sourcename', 'stun_duration', 'stuns', 'targethero', 'targetillusion',
       'targetname', 'targetsourcename', 'targetteam', 'team',
       'teamfight_participation', 'tick', 'time', 'towers_killed', 'type',
       'unit', 'unit_list', 'value', 'valuename', 'vec_x', 'vec_y', 'x', 'xp',
       'xp_reason', 'y', 'z'],
      dtype='object')

assists, kills, time, ?gold?

In [79]:
for t in data[data['kills']>0]['kills'].dropna().unique():
    for h in data[data['kills']>0]['hero_id'].dropna().unique():
        tmp = data[(data['kills'] == t)& (data['hero_id'] == h)][['assists', 'kills', 'time','hero_id']].dropna().sort_values(by=['time']).head(1)
        if len(tmp) == 1:
            print (tmp)

        assists  kills     time  hero_id
127988      0.0    1.0  889.871     34.0
        assists  kills      time  hero_id
163329      1.0    1.0  959.8539    107.0
        assists  kills       time  hero_id
213365      0.0    1.0  1060.8959      2.0
        assists  kills       time  hero_id
215900      0.0    1.0  1065.8947     79.0
        assists  kills       time  hero_id
260516      0.0    1.0  1155.3395     50.0
        assists  kills       time  hero_id
326627      0.0    1.0  1288.8402     66.0
        assists  kills      time  hero_id
514652      0.0    1.0  1666.348     75.0
        assists  kills       time  hero_id
520470      2.0    1.0  1677.3453     16.0
        assists  kills       time  hero_id
608454      2.0    1.0  1854.8353    109.0
        assists  kills       time  hero_id
746986      3.0    1.0  2129.3987     48.0
        assists  kills       time  hero_id
184930      1.0    2.0  1002.8434     34.0
        assists  kills       time  hero_id
771890      7.0    

In [75]:
Combat[Combat['time']<=889.871]

,attackername,targetname,time,location
83671,npc_dota_earth_spirit_stone,npc_dota_earth_spirit_stone,802.49230,7.0
92754,npc_dota_earth_spirit_stone,npc_dota_earth_spirit_stone,819.75476,4.0
98731,npc_dota_hero_tinker,npc_dota_creep_goodguys_ranged,831.01870,2.0
99486,npc_dota_hero_silencer,npc_dota_creep_badguys_melee,832.48500,2.0
101596,npc_dota_hero_tinker,npc_dota_creep_goodguys_melee,836.81726,2.0
102792,npc_dota_hero_terrorblade,npc_dota_creep_goodguys_melee,839.25000,3.0
103352,npc_dota_hero_tinker,npc_dota_creep_goodguys_melee,840.38306,2.0
103478,npc_dota_hero_luna,npc_dota_creep_badguys_melee,840.68300,1.0
105067,npc_dota_creep_badguys_ranged,npc_dota_creep_goodguys_melee,844.14880,2.0
105352,npc_dota_hero_luna,npc_dota_creep_goodguys_melee,844.71533,1.0


In [52]:
data[data['kills']>0][['assists', 'kills', 'time','hero_id']].dropna()

,assists,kills,time,hero_id
127988,0.0,1.0,889.87100,34.0
128232,0.0,1.0,890.33750,34.0
128517,0.0,1.0,890.87070,34.0
128755,0.0,1.0,891.33730,34.0
129026,0.0,1.0,891.87050,34.0
129259,0.0,1.0,892.33704,34.0
129515,0.0,1.0,892.87024,34.0
129730,0.0,1.0,893.33680,34.0
130009,0.0,1.0,893.87000,34.0
130236,0.0,1.0,894.33655,34.0
